## DMM Result

In [100]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
DMM = pickle.load(open('../../model_lda/DMM_result.pkl','rb'))

data = pickle.load(open('preprocessed_review.pickle', 'rb'))
data_join = [' '.join(doc) for doc in data]
cv = CountVectorizer()
X = cv.fit_transform(data_join).toarray()

DMM_lam = [DMM.components_[k,:] for k in range(4)]
def print_top_words(lam, feature_names, n_top_words):
    for topic_id, topic in enumerate(lam):
        print('\nTopic Nr.%d:' % int(topic_id + 1))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
                       + ' | ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
print_top_words(DMM_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
hotel 3595.78 | stay 2803.74 | room 2764.67 | staff 1623.16 | great 1596.15 | service 1074.03 | waikiki 1015.44 | location 961.98 | beach 859.0 | place 831.88 | 

Topic Nr.2:
hotel 4903.6 | room 4674.14 | beach 3114.4 | stay 3002.51 | waikiki 2391.2 | view 2289.3 | great 2059.07 | nice 1425.27 | pool 1413.63 | location 1318.82 | 

Topic Nr.3:
hotel 7295.34 | great 5695.8 | room 4682.51 | stay 4162.87 | location 3840.33 | beach 3514.46 | staff 3322.45 | good 2555.8 | waikiki 2407.07 | nice 2009.29 | 

Topic Nr.4:
room 2199.08 | stay 2100.28 | hotel 1505.68 | staff 1337.29 | great 1107.37 | view 1024.0 | beach 826.53 | time 807.45 | waikiki 806.7 | location 740.27 | 


In [69]:
words_, a, c = top_20_words_for_topics(DMM_lam)
a

0.525

## CLDA Result

In [70]:
CLDA = pickle.load(open('../../model_lda/CLDA_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(4)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
get 3589.1 | price 2652.0 | go 2405.05 | time 2400.06 | day 2306.27 | resort 2213.1 | value 1917.0 | pay 1873.0 | fee 1791.0 | just 1709.18 | 

Topic Nr.2:
staff 8591.0 | pool 4981.0 | service 4875.0 | friendly 4705.0 | helpful 3495.0 | check 3270.0 | stay 3206.75 | desk 2724.0 | make 2513.1 | parking 2103.0 | 

Topic Nr.3:
great 10297.07 | beach 8314.1 | location 6861.1 | waikiki 6593.93 | restaurant 4208.0 | breakfast 3596.0 | walk 3320.1 | good 2879.16 | food 2667.0 | place 2579.82 | 

Topic Nr.4:
hotel 17017.05 | room 15320.0 | stay 9862.25 | view 5256.1 | nice 4207.09 | clean 3346.04 | close 3118.0 | front 3069.0 | ocean 2423.1 | bed 2215.1 | 


## CDMM Result (two phase)

In [106]:
CLDA = pickle.load(open('../../model_lda/CDMM_two_phase_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(5)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 20)


Topic Nr.1:
pay 1079.08 | charge 1070.03 | fee 1069.21 | price 1064.93 | rate 1034.0 | cost 1033.74 | money 1025.6 | value 1023.94 | expensive 1020.05 | cheap 1007.0 | cheaper 1005.0 | reasonably 1004.9 | pricey 1004.0 | paid 1001.0 | fees 1001.0 | priced 1000.0 | room 442.94 | hotel 371.14 | stay 205.2 | one 152.11 | 

Topic Nr.2:
room 5021.83 | hotel 4718.54 | beach 3236.78 | stay 2963.78 | staff 2933.38 | great 2686.82 | pool 2483.51 | view 2437.29 | service 2334.06 | waikiki 2330.47 | friendly 2090.22 | helpful 1852.73 | location 1745.25 | check 1671.38 | nice 1580.42 | parking 1539.6 | desk 1531.49 | offer 1410.86 | valet 1372.82 | clean 1258.58 | 

Topic Nr.3:
hotel 3275.77 | great 2881.5 | room 2242.14 | restaurant 1922.88 | location 1905.46 | stay 1757.62 | beach 1712.54 | breakfast 1669.84 | good 1452.04 | food 1451.32 | coffee 1382.1 | staff 1375.04 | water 1322.32 | drink 1155.24 | tea 1120.52 | fruit 1116.91 | starbucks 1114.45 | waikiki 1095.58 | dinner 1092.39 | cafe 108

In [107]:
import pandas as pd
import numpy as np
from collections import Counter
def top_20_words_for_topics(CLDA_lam, n=50):
    K = len(CLDA_lam)
    words = {}
    unique_words = []
    for k in range(K):
        CLDA_lam_word = pd.DataFrame({'word':cv.get_feature_names(), 'lam':CLDA.components_[k,:] }).sort_values(by='lam', ascending=False)
        top_words = CLDA_lam_word.iloc[:n,:]['word'].tolist()
        words[k] = CLDA_lam_word.iloc[:n,:].set_index('word').to_dict()['lam']
        unique_words += top_words
        
    c = Counter(unique_words)
    words_all_appear = []
    for k,v in c.items():
        if v == K:
            words_all_appear.append(k)
    
    
    
    words_degree = []
    for k in range(K):
        for w,d in words[k].items():
            if w in words_all_appear:
                words_degree.append(d)
        
    return words, len(set(unique_words)) / (K*n), len(set(unique_words)) , np.mean(words_degree)
words_, a, b, c = top_20_words_for_topics(CLDA_lam)
a, b

(0.525, 105)

## CDMM Result (one phase)

In [108]:
CLDA = pickle.load(open('../../model_lda/CDMM_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(4)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 50)


Topic Nr.1:
hotel 1321.47 | pay 1191.55 | price 1188.85 | fee 1167.34 | room 1163.75 | charge 1163.58 | value 1101.36 | rate 1097.24 | money 1092.0 | cost 1077.69 | expensive 1061.99 | cheap 1032.32 | cheaper 1026.83 | reasonably 1014.74 | pricey 1010.34 | fees 1001.0 | paid 1001.0 | priced 1000.98 | stay 817.48 | great 378.63 | one 368.82 | staff 347.67 | get 335.53 | location 324.85 | place 315.03 | check 306.55 | waikiki 298.18 | beach 297.35 | night 265.62 | clean 257.63 | view 257.57 | good 257.1 | service 254.54 | book 244.36 | go 241.4 | bed 226.51 | make 220.14 | will 219.55 | also 212.14 | day 207.9 | pool 203.26 | time 198.02 | nice 193.38 | just 185.42 | desk 182.73 | even 176.97 | front 175.65 | resort 163.6 | walk 155.12 | back 154.51 | 

Topic Nr.2:
hotel 8814.65 | room 7302.46 | stay 6273.04 | staff 4934.48 | great 4729.18 | beach 4581.76 | waikiki 3944.91 | service 3477.87 | view 3319.92 | pool 3156.33 | friendly 2916.09 | location 2867.34 | helpful 2370.82 | check 232

In [109]:
words_, a, b, c = top_20_words_for_topics(CLDA_lam,n=50)
a, b

(0.6066666666666667, 91)

In [88]:
dir_ = "/Users/shinbo/Desktop/metting/LDA/0. data/20news-bydate/newsgroup_preprocessed_corpus.pickle"
data = pickle.load(open(dir_, 'rb'))
data_join = [' '.join(doc) for doc in data]
cv = CountVectorizer()
X = cv.fit_transform(data_join).toarray()

In [89]:
model = pickle.load(open('../../model_lda/LDA_PAPER_result.pkl','rb'))
model_lam =  [model.components_[k,:] for k in range(10)]
print_top_words(model_lam, list(cv.get_feature_names()), 50)


Topic Nr.1:
get 379.19 | use 360.6 | university 318.36 | know 285.97 | like 283.16 | think 254.44 | make 228.8 | go 226.09 | people 196.36 | good 191.53 | time 190.17 | well 182.12 | work 176.68 | system 175.63 | problem 173.05 | writes 169.34 | see 168.37 | drive 165.27 | want 163.83 | cx 162.89 | need 160.0 | distribution 157.39 | new 157.08 | also 153.9 | look 147.29 | anyone 146.77 | ww 145.64 | thing 141.17 | window 140.65 | hz 136.57 | take 135.99 | way 131.52 | much 129.24 | uw 128.67 | computer 128.43 | year 127.94 | please 125.01 | thanks 124.9 | right 123.9 | card 123.68 | try 122.95 | find 122.37 | come 122.34 | even 122.31 | give 116.61 | question 116.6 | c_ 116.16 | run 113.36 | help 112.49 | two 109.94 | 

Topic Nr.2:
go 600.02 | get 530.75 | know 520.64 | people 517.17 | like 413.69 | use 403.84 | think 400.8 | time 356.53 | take 351.59 | come 342.32 | make 323.45 | see 306.27 | well 304.21 | drive 299.63 | university 270.6 | even 258.71 | want 256.69 | tell 256.51 | th

In [3]:
model = pickle.load(open('../../model_lda/LDA_PAPER_result.pkl','rb'))

In [4]:
model.kappa_1

array([[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]])

0.49999999999954525